In [244]:
import numpy as np
import pandas as pd
import csv
import json
from sklearn import preprocessing

In [245]:
'''
ID - an Id that represents a (Shop, Item) tuple within the test set
shop_id - unique identifier of a shop
item_id - unique identifier of a product
item_category_id - unique identifier of item category
item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
item_price - current price of an item
date - date in format dd/mm/yyyy
date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
item_name - name of item
shop_name - name of shop
item_category_name - name of item category
'''

'\nID - an Id that represents a (Shop, Item) tuple within the test set\nshop_id - unique identifier of a shop\nitem_id - unique identifier of a product\nitem_category_id - unique identifier of item category\nitem_cnt_day - number of products sold. You are predicting a monthly amount of this measure\nitem_price - current price of an item\ndate - date in format dd/mm/yyyy\ndate_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33\nitem_name - name of item\nshop_name - name of shop\nitem_category_name - name of item category\n'

In [246]:
items = pd.read_csv("../data/items.csv")
item_cat = pd.read_csv("../data/item_categories.csv")
train = pd.read_csv("../data/sales_train.csv")
sample_submission = pd.read_csv("../data/sample_submission.csv")
shops = pd.read_csv("../data/shops.csv")
test = pd.read_csv("../data/test.csv")

In [247]:
print('items : ', len(items))
print('item_cat : ', len(item_cat))
print('train : ' , len(train))
print('test : ', len(test))
print('shops : ', len(shops))

items :  22170
item_cat :  84
train :  2935849
test :  214200
shops :  60


In [248]:
print(min(train['item_cnt_day']), max(train['item_cnt_day']))

-22.0 2169.0


In [249]:
train['date'] = train['date'].str[3:]

In [254]:
train_groupby = train.groupby(['shop_id', 'item_id', 'date', 'item_price', 'date_block_num'])['item_cnt_day'].agg('sum').reset_index()

In [255]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,01.2013,0,59,22154,999.00,1.0
1,01.2013,0,25,2552,899.00,1.0
2,01.2013,0,25,2552,899.00,-1.0
3,01.2013,0,25,2554,1709.05,1.0
4,01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.2015,33,25,7409,299.00,1.0
2935845,10.2015,33,25,7460,299.00,1.0
2935846,10.2015,33,25,7459,349.00,1.0
2935847,10.2015,33,25,7440,299.00,1.0


In [ ]:
train = train_groupby

In [278]:
train

,shop_id,item_id,date,item_price,date_block_num,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,#3_amount,#4_amount,month,year,item_price_avg
0,0,30,02.2013,265.0,1,31.0,0.036641,8215.0,0.026655,40,1.000000,0.011191,2,2013,845.775122
1,1,30,02.2013,265.0,1,10.0,0.019266,2650.0,0.012181,40,1.000000,0.011191,2,2013,845.775122
2,2,30,03.2013,359.0,2,1.0,0.097569,359.0,0.185965,40,1.000000,0.011191,3,2013,845.775122
3,2,30,04.2014,169.0,15,1.0,0.097569,169.0,0.185965,40,1.000000,0.011191,4,2014,845.775122
4,2,30,05.2014,169.0,16,1.0,0.097569,169.0,0.185965,40,1.000000,0.011191,5,2014,845.775122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739017,55,13093,10.2015,250.0,33,4.0,0.203120,1000.0,0.210418,36,0.000033,0.000123,10,2015,845.775122
1739018,55,13094,09.2015,2500.0,32,3.0,0.203120,7500.0,0.210418,36,0.000033,0.000075,9,2015,845.775122
1739019,55,13095,08.2015,500.0,31,1.0,0.203120,500.0,0.210418,36,0.000033,0.000085,8,2015,845.775122
1739020,55,13095,09.2015,500.0,32,1.0,0.203120,500.0,0.210418,36,0.000033,0.000085,9,2015,845.775122


In [258]:
train_groupby_shop = train_groupby.groupby(['shop_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_item = train_groupby.groupby(['item_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_shop_and_item = train_groupby.groupby(['shop_id', 'item_id','date'])['item_cnt_day'].agg('sum').reset_index()

In [259]:
train_groupby_item

,item_id,item_cnt_day
0,0,1.0
1,1,6.0
2,2,2.0
3,3,2.0
4,4,1.0
...,...,...
21802,22165,2.0
21803,22166,348.0
21804,22167,1320.0
21805,22168,6.0


In [260]:
## train_groupby_item에서 item_cnt_day를 #2_amount로 이름 변경
train_groupby_shop = train_groupby_shop.rename(columns = {'item_cnt_day': '#2_amount'}, inplace = False)
train = pd.merge(train, train_groupby_shop, on='shop_id')

In [261]:
train['tmp'] = train['item_price']*train['item_cnt_day']
train_groupby_shop_price = train.groupby(['shop_id'])['tmp'].agg('sum').reset_index()
train_groupby_shop_price = train_groupby_shop_price.rename(columns = {'tmp': '#2_price'}, inplace = False)
train = pd.merge(train, train_groupby_shop_price, on ='shop_id')

In [262]:
train = pd.merge(train, items[['item_id', 'item_category_id']], on = 'item_id')

In [263]:
train_groupby_item_category = train.groupby(['item_category_id'])['item_cnt_day'].agg('sum').reset_index()

In [264]:
train = pd.merge(train, train_groupby_item_category , on='item_category_id')

In [265]:
train = train.rename(columns = {'item_cnt_day_x': 'item_cnt_day'}, inplace = False)
train = train.rename(columns = {'item_cnt_day_y': '#3_amount'}, inplace = False)

In [266]:
train_groupby_item = train_groupby_item.rename(columns = {'item_cnt_day' : '#4_amount'}, inplace = False)

In [267]:
train = pd.merge(train, train_groupby_item , on='item_id')

In [268]:
#train = train.drop(['item_cnt_day_x'], axis = 1)

In [269]:
train['month'] = train['date'].str[:2]

In [270]:
train['year'] = train['date'].str[3:]

In [271]:
def normalize_column(df, column_name):
    x = df[[column_name]]
    # Create x, where x the 'scores' column's values as floats
    x = x.values.astype(float)
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)
    # Run the normalizer on the dataframe
    df_normalized = pd.DataFrame(x_scaled, columns = [column_name])
    
    df[column_name] = df_normalized
    
    return df

In [272]:
normalize_column(train, '#2_amount')
normalize_column(train, '#2_price')
normalize_column(train, '#3_amount')
normalize_column(train, '#4_amount')
train['month'] = train['month'].astype(int)
train['year'] = train['year'].astype(int)

In [273]:
train.drop('tmp', axis = 1)

,shop_id,item_id,date,item_price,date_block_num,item_cnt_day,#2_amount,#2_price,item_category_id,#3_amount,#4_amount,month,year
0,0,30,02.2013,265.0,1,31.0,0.036641,0.026655,40,1.000000,0.011191,2,2013
1,1,30,02.2013,265.0,1,10.0,0.019266,0.012181,40,1.000000,0.011191,2,2013
2,2,30,03.2013,359.0,2,1.0,0.097569,0.185965,40,1.000000,0.011191,3,2013
3,2,30,04.2014,169.0,15,1.0,0.097569,0.185965,40,1.000000,0.011191,4,2014
4,2,30,05.2014,169.0,16,1.0,0.097569,0.185965,40,1.000000,0.011191,5,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739017,55,13093,10.2015,250.0,33,4.0,0.203120,0.210418,36,0.000033,0.000123,10,2015
1739018,55,13094,09.2015,2500.0,32,3.0,0.203120,0.210418,36,0.000033,0.000075,9,2015
1739019,55,13095,08.2015,500.0,31,1.0,0.203120,0.210418,36,0.000033,0.000085,8,2015
1739020,55,13095,09.2015,500.0,32,1.0,0.203120,0.210418,36,0.000033,0.000085,9,2015


# 6.1

In [274]:
train['item_price_avg'] = train['item_price'].mean()

# 6.2

In [275]:
out = []
unique_cat = train['item_category_id'].unique()
for cat in unique_cat:
    same_cat_diff_shop = train[train['item_category_id']==cat]
    same_shop = same_cat_diff_shop.drop_duplicates(subset='shop_id', keep="last") #unique shops
    norm=(same_shop['item_price']-same_shop['item_price'].mean())/same_shop['item_price'].std()
    same_shop['item_cat_price_norm'] = norm
    arr = np.array(same_shop)[:,[2,8, -1]]
    out.append(arr)
    #out.append(same_shop['date','item_category_id' ,'item_cat_price_norm'])
    #train = train.merge(same_shop, on = list(train.columns), how = 'left')

/opt/anaconda3/envs/jaeyoung/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [276]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,#3_amount,#4_amount,month,year,item_price_avg,item_cat_price_norm
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,0.32055,0.000373,1,2013,890.853233,NaN
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,NaN
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,NaN
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,NaN
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,NaN


In [277]:
out = np.array(out)
v = np.vstack(out)
df = pd.DataFrame(v, columns = ['shop_id', 'item_category_id', 'item_cat_price_norm'])
df = df.fillna(0)
df_train = train.merge(df, on = ['shop_id', 'item_category_id'], how = 'left')
# train = df_train.drop(['item_cat_price_norm_x'], axis = 1)
train = df_train

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
train.head()

# 1

In [112]:
train = pd.concat([train,pd.get_dummies(train['shop_id'], prefix='shop_id')],axis=1)

In [113]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,...,shop_id_50,shop_id_51,shop_id_52,shop_id_53,shop_id_54,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,0,0,0,1
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,...,0,0,0,0,0,1,0,0,0,0
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,...,0,0,0,0,0,1,0,0,0,0
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,...,0,0,0,0,0,1,0,0,0,0
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,...,0,0,0,0,0,1,0,0,0,0


## 8

In [114]:
lag2015 = train[train['date'] == '10.2015']['item_cnt_day']
lag2013 = train[train['date'] == '11.2013']['item_cnt_day']
lag2014 = train[train['date'] == '11.2014']['item_cnt_day']

In [115]:
lag2013.shape, lag2014.shape, lag2015.shape

((96736,), (86428,), (53514,))

In [116]:
lag2013_ = lag2013.to_frame()
lag2014_ = lag2014.to_frame()
lag2015_ = lag2015.to_frame()

temp2 = pd.concat([lag2013_, lag2014_,lag2015_], axis=1).fillna(0)
temp2.columns=['2013.11','2014.11','2015.10']
temp2

,2013.11,2014.11,2015.10
9,0.0,0.0,1.0
14,0.0,0.0,1.0
25,0.0,0.0,1.0
31,0.0,0.0,1.0
45,0.0,0.0,1.0
...,...,...,...
2935841,0.0,0.0,1.0
2935842,0.0,0.0,1.0
2935843,0.0,0.0,1.0
2935844,0.0,0.0,1.0


In [117]:
train = pd.concat([train, temp2], axis=1).fillna(0)
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,...,shop_id_53,shop_id_54,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59,2013.11,2014.11,2015.10
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,1,0.0,0.0,0.0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,...,0,0,1,0,0,0,0,0.0,0.0,1.0
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,...,0,0,1,0,0,0,0,0.0,0.0,0.0
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,...,0,0,1,0,0,0,0,0.0,0.0,0.0
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,...,0,0,1,0,0,0,0,0.0,0.0,0.0


## 9 

In [118]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [119]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,...,shop_id_53,shop_id_54,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59,2013.11,2014.11,2015.10
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,1,0.0,0.0,0.0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [120]:
from sklearn.feature_extraction.text import CountVectorizer

item_name_list = items['item_name']

vect = CountVectorizer(analyzer='word', max_features=20)
bow_feature = vect.fit_transform(item_name_list).toarray()
bow_feature

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [121]:
type(bow_feature)

numpy.ndarray

In [122]:
bow_df = pd.DataFrame(bow_feature)
bow_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22165,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
22166,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
22167,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22168,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [123]:
item_ids = train['item_id']
item_ids

0          22154
1          22154
2          22154
3          22154
4          22154
           ...  
2935844    13093
2935845    13091
2935846    13094
2935847    13094
2935848    13092
Name: item_id, Length: 2935849, dtype: int64

In [124]:
temp = bow_df.iloc[item_ids].reset_index(drop=True)
temp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935845,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935846,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935847,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [125]:
train_ = pd.concat([train, temp], axis=1)
train_

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,...,10,11,12,13,14,15,16,17,18,19
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,0,0,0,0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1


## TEST!!

In [284]:
test['month'] = 11
test['year'] = 2015
test['data_block_num'] =34

In [285]:
test = pd.merge(test,items, on = 'item_id')

In [286]:
test.drop('item_name', axis =1)

,ID,shop_id,item_id,month,year,data_block_num,item_category_id
0,0,5,5037,11,2015,34,19
1,5100,4,5037,11,2015,34,19
2,10200,6,5037,11,2015,34,19
3,15300,3,5037,11,2015,34,19
4,20400,2,5037,11,2015,34,19
...,...,...,...,...,...,...,...
214195,193799,46,969,11,2015,34,37
214196,198899,41,969,11,2015,34,37
214197,203999,44,969,11,2015,34,37
214198,209099,39,969,11,2015,34,37


In [287]:
train_1 = train[train['date'] == '10.2015'][['item_id', 'item_price']]

In [288]:
train_1

,item_id,item_price
283,30,129.0
557,32,149.0
589,32,149.0
610,32,149.0
649,32,149.0
...,...,...
1738996,13098,2000.0
1739010,13099,500.0
1739014,13092,2000.0
1739017,13093,250.0


In [289]:
# df.join(other, lsuffix='_caller', rsuffix='_other')
test2 = test.join(train_1, lsuffix = '_caller', rsuffix = '_other', how='left')

In [290]:
test2 = test2.drop( 'item_id_other', axis=1)
test2 = test2.drop( 'item_price', axis=1)
test2 = test2.drop( 'item_name', axis=1)
test2 = test2.rename(columns = {'item_id_caller': 'item_price'}, inplace = False)

In [291]:
#nan 개수
np.count_nonzero(np.isnan(np.array(test2['item_price'])))

0

In [292]:
train_1 = train[train['date'] == '09.2015'][['item_id', 'item_price']]

In [293]:
test2

,ID,shop_id,item_price,month,year,data_block_num,item_category_id
0,0,5,5037,11,2015,34,19
1,5100,4,5037,11,2015,34,19
2,10200,6,5037,11,2015,34,19
3,15300,3,5037,11,2015,34,19
4,20400,2,5037,11,2015,34,19
...,...,...,...,...,...,...,...
214195,193799,46,969,11,2015,34,37
214196,198899,41,969,11,2015,34,37
214197,203999,44,969,11,2015,34,37
214198,209099,39,969,11,2015,34,37
